# Анализ тональности (Women’s Clothing Reviews) — RU

В этом ноутбуке:
1) Препроцессинг датасета до формата `text,label` (label: 0 негатив, 1 позитив) по `Rating`.
2) Перевод EN→RU (Helsinki-NLP/opus-mt-en-ru).
3) Обучение **трёх семейств** моделей на русском тексте:
   - Classic (sklearn): Logistic Regression / Linear SVM (calibrated) / Naive Bayes
   - RNN (PyTorch): BiLSTM
   - Transformer: RuBERT fine-tuning
4) Метрики и confusion matrix по test:
   - Accuracy, Macro-F1, Weighted-F1, ROC-AUC, PR-AUC

Запуск BERT/перевода может требовать интернет для скачивания моделей и/или GPU для скорости.


In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

RAW_PATH = "../data/raw/Womens Clothing E-Commerce Reviews.csv"
PROC_EN = "../data/processed/dataset_en.csv"
PROC_RU = "../data/processed/dataset_ru.csv"


## 1) Препроцессинг: оставляем только text,label по Rating

In [4]:
from src.preprocess_womens_dataset import main as _preprocess_main

# Здесь просто показываем ожидаемый результат.

df_raw = pd.read_csv(RAW_PATH)
out = df_raw[["Review Text", "Rating"]].copy().dropna(subset=["Review Text"]) 
out["Rating"] = pd.to_numeric(out["Rating"], errors="coerce")
out = out.dropna(subset=["Rating"]) 
out = out[out["Rating"].between(1,5)]
out["text"] = out["Review Text"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
out["label"] = (out["Rating"] >= 4).astype(int)
df_en = out[["text","label"]]

df_en.to_csv(PROC_EN, index=False)
print("Saved", PROC_EN, "rows=", len(df_en))
print(df_en["label"].value_counts())
df_en.head()

Saved ../data/processed/dataset_en.csv rows= 22641
label
1    17448
0     5193
Name: count, dtype: int64


,text,label
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happened ...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


## 2) Перевод EN→RU

In [6]:
from transformers import pipeline

translator = pipeline(
    "translation_en_to_ru",
    model="Helsinki-NLP/opus-mt-en-ru",
    device=-1,  # CPU
)


def translate_batch(texts, batch_size=64):
    ru = []
    for i in range(0, len(texts), batch_size):
        res = translator(
            texts[i:i+batch_size],
            batch_size=batch_size,
            max_length=256,   # ограничить длину результата
            num_beams=1
        )
        ru.extend([r["translation_text"] for r in res])
        if i == 0 or (i // batch_size) % 50 == 0:
            print(f"Translated {len(ru)}/{len(texts)}")
    return ru


# Для демонстрации можно ограничить max_rows
max_rows = 5000  # 0 для полного датасета
work = df_en.copy()
if max_rows:
    work = work.head(max_rows)

work["text_ru"] = translate_batch(work["text"].tolist(), batch_size=16)
work.to_csv(PROC_RU, index=False)
print("Saved", PROC_RU)
work.head()

Device set to use cpu


Translated 16/5000
Translated 816/5000
Translated 1616/5000
Translated 2416/5000
Translated 3216/5000
Translated 4016/5000
Translated 4816/5000
Saved ../data/processed/dataset_ru.csv


,text,label,text_ru
0,Absolutely wonderful - silky and sexy and comf...,1,"Абсолютно чудесно - шелк, сексуально и комфортно"
1,Love this dress! it's sooo pretty. i happened ...,1,"Я так и нахожу его в магазине, и я рада, что я..."
2,I had such high hopes for this dress and reall...,0,"Я так надеялся на это платье и хотел, чтобы он..."
3,"I love, love, love this jumpsuit. it's fun, fl...",1,"Я люблю, люблю этот комбинезон, это весело, фл..."
4,This shirt is very flattering to all due to th...,1,Эта рубашка очень лестна для всех из-за регули...


## 3) Split train/test

In [6]:
df = pd.read_csv(PROC_RU)

X = df["text_ru"].astype(str).values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# далее ещё сделаем val из train для RNN/BERT
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

print("Train:", len(X_tr), "Val:", len(X_val), "Test:", len(X_test))

Train: 3600 Val: 400 Test: 1000


## 4) Classic модели (sklearn)

In [7]:
from src.text_models.classic import train_classic_models, predict_classic
from src.text_models.metrics import evaluate_binary

bundle = train_classic_models(X_tr, y_tr)

results = {}
for name in bundle.models.keys():
    y_pred, y_prob = predict_classic(bundle, X_test, name)
    m = evaluate_binary(y_test, y_pred, y_prob)
    results[f"classic/{name}"] = m
    print(name, m.as_dict())


logreg {'accuracy': 0.84, 'macro_f1': 0.7837545614272199, 'weighted_f1': 0.843749695904852, 'roc_auc': 0.8987478411053542, 'pr_auc': 0.967593728023771, 'confusion': [[165, 63], [97, 675]]}
svm_linear {'accuracy': 0.856, 'macro_f1': 0.7902097902097902, 'weighted_f1': 0.8541202797202797, 'roc_auc': 0.9061108081083538, 'pr_auc': 0.970187494742422, 'confusion': [[148, 80], [64, 708]]}
naive_bayes {'accuracy': 0.786, 'macro_f1': 0.5141840107515164, 'weighted_f1': 0.7118683665685954, 'roc_auc': 0.8501954367784746, 'pr_auc': 0.9481679303960161, 'confusion': [[19, 209], [5, 767]]}


## 5) RNN: BiLSTM (PyTorch)

In [8]:
from src.text_models.rnn import train_rnn, predict_rnn
from src.text_models.metrics import evaluate_binary

rnn_bundle = train_rnn(
    x_train=list(X_tr), y_train=y_tr,
    x_val=list(X_val), y_val=y_val,
    epochs=3
)

y_pred, y_prob = predict_rnn(rnn_bundle, list(X_test))
m = evaluate_binary(y_test, y_pred, y_prob)
results["rnn/bilstm"] = m
print(m.as_dict())


Epoch 1: train_loss=0.5519, val_acc=0.7725
Epoch 2: train_loss=0.5189, val_acc=0.7575
Epoch 3: train_loss=0.4962, val_acc=0.7450
{'accuracy': 0.749, 'macro_f1': 0.45754477416789313, 'weighted_f1': 0.6738498803788974, 'roc_auc': 0.502130488137442, 'pr_auc': 0.7702986966468642, 'confusion': [[8, 220], [31, 741]]}


## 6) Transformer: RuBERT fine-tuning

In [9]:
from src.text_models.bert import finetune_bert, predict_bert
from src.text_models.metrics import evaluate_binary

bert_bundle = finetune_bert(
    x_train=list(X_tr), y_train=y_tr,
    x_val=list(X_val), y_val=y_val,
    model_name="DeepPavlov/rubert-base-cased",
    epochs=1,
    out_dir="../models/bert_tmp",
    
)

y_pred, y_prob = predict_bert(bert_bundle, list(X_test))
m = evaluate_binary(y_test, y_pred, y_prob)
results["bert/rubert"] = m
print(m.as_dict())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.315600,0.372437


{'accuracy': 0.881, 'macro_f1': 0.815099962553858, 'weighted_f1': 0.8751495779151667, 'roc_auc': 0.9389203254249614, 'pr_auc': 0.9812602425879718, 'confusion': [[142, 86], [33, 739]]}


## 7) Сводная таблица + выбор лучшей модели

In [11]:
import pandas as pd

rows = []
for name, m in results.items():
    d = m.as_dict()
    rows.append({
        "model": name,
        "accuracy": d["accuracy"],
        "macro_f1": d["macro_f1"],
        "weighted_f1": d["weighted_f1"],
        "roc_auc": d["roc_auc"],
        "pr_auc": d["pr_auc"],
        "cm": list(d["confusion"]),
    })

df_res = pd.DataFrame(rows).sort_values("macro_f1", ascending=False)
df_res

,model,accuracy,macro_f1,weighted_f1,roc_auc,pr_auc,cm
4,bert/rubert,0.881,0.815100,0.875150,0.938920,0.981260,"[[142, 86], [33, 739]]"
1,classic/svm_linear,0.856,0.790210,0.854120,0.906111,0.970187,"[[148, 80], [64, 708]]"
0,classic/logreg,0.840,0.783755,0.843750,0.898748,0.967594,"[[165, 63], [97, 675]]"
2,classic/naive_bayes,0.786,0.514184,0.711868,0.850195,0.948168,"[[19, 209], [5, 767]]"
3,rnn/bilstm,0.749,0.457545,0.673850,0.502130,0.770299,"[[8, 220], [31, 741]]"


## 8) Сохранение лучшей модели (обычно RuBERT)

In [12]:
# Обычно для деплоя выбираем RuBERT. Сохраняем в ../models/bert/

import shutil
import os

out_dir = "../models/bert"
os.makedirs(out_dir, exist_ok=True)

# сохраняем tokenizer+model в формате HuggingFace
bert_bundle.tokenizer.save_pretrained(out_dir)
bert_bundle.model.save_pretrained(out_dir)
print("Saved HF model to", out_dir)

Saved HF model to ../models/bert


## 9) Анализ ошибок: примеры false positive/false negative

In [14]:
# Для анализа ошибок используем предсказания BERT

import numpy as np

y_pred = np.array(y_pred)

fp_idx = np.where((y_test == 0) & (y_pred == 1))[0]
fn_idx = np.where((y_test == 1) & (y_pred == 0))[0]

print("False Positive:", len(fp_idx))
print("False Negative:", len(fn_idx))

print("Примеры FP:")
for i in fp_idx[:5]:
    print("-", X_test[i])

print("Примеры FN:")
for i in fn_idx[:5]:
    print("-", X_test[i])


False Positive: 86
False Negative: 33
Примеры FP:
- Обычно я не пересматриваю свои покупки, но я был так удивлен тем, как плохо было сделано это платье, что я не мог удержаться, кроме как выставить обзор. Линия шеи даже не загибается, так что она замкнулась. материал тонкий и дешевый. Это платье даже не стоит 20 долларов, по моему мнению. Я ожидал хорошо сделанное, хорошее платье для высокой цены.
- Я внимательно прочёл инструкции по уборке, следовал за ними, и все же краситель побежал и покрасил некоторые из моих цветных вещей. Пи** вытащил меня! Сразу после первой стирки и даже перед тем, как надеть их, у меня на штанах были неровные цвета. Плохой материал или краситель - вот что вызывает это. Теперь мне нужно пойти и посмотреть, смогу ли я достать свою другую вспахнутую одежду. Спасибо большое розничному торговцу.
- Люблю синий цвет, и кусочек искры дает этому случайному кусочку немного причуды. Круто. Люблю большие наручники. Но я выглядел как колокол свободы в маленьком. Он огромн